---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="453d324d" -->
# Table des matières
1. [Qu'est-ce que le biais et la variance?](#quest-ce-que-le-biais-et-la-variance)
1. [Comment calculer le biais et la variance en régression?](#comment-calculer-le-biais-et-la-variance-en-régression)
1. [La relation entre le MSE, le biais et la variance](#la-relation-entre-le-mse-le-biais-et-la-variance)
    1. [Calcul du MSE, du $\text{biais}^2$ et de la variance](#calcul-du-mse-du-biais-et-de-la-variance)
      1. [Comparaison des mesures du MSE avec la relation théorique](#comparaison-des-mesures-du-mse-avec-la-relation-théorique)
1. [Quel est le polynôme recalant le mieux les données?](#quel-est-le-polynôme-recalant-le-mieux-les-données)
1. [Comparaison des courbes du MSE en entraînement et en test](#comparaison-des-courbes-du-mse-en-entraînement-et-en-test)
1. [Exemples de sous-apprentissage et de surapprentissage en régression](#exemples-de-sous-apprentissage-et-de-surapprentissage-en-régression)
    1. [Sous-apprentissage](#sous-apprentissage)
    1. [Surapprentissage](#surapprentissage)
    1. [Le juste milieu](#le-juste-milieu)
1. [Le sous-apprentissage et le surapprentissage en classification](#le-sous-apprentissage-et-le-surapprentissage-en-classification)
<!-- #endregion -->



In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

sns.set(color_codes=True)

seed = 42
np.random.seed(seed)



<!-- #region id="3f16da2d" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/under-vs-over-training.jpeg"  width="600" />
    <div>
    <font size="1.5">Image Source: https://datacadamia.com/data_mining/overfitting</font>
    </div>
</div>

<!-- #endregion -->

<!-- #region id="beb23636" -->
Dans ce module, nous allons discuter des phénomènes de sous-apprentissage (*Underfitting*) et de surapprentissage (*Overfitting*) que l'on observe
régulièrement en apprentissage automatique. Nous allons aborder le sujet en nous concentrant sur
la régression et allons utiliser des modèles polynomiaux pour recaler des données expérimentales. Ils sont faciles à
utiliser et les deux phénomènes sont faciles à visualiser avec ceux-ci.

Pour ce faire, nous allons discuter de la métrique de l'erreur quadratique moyenne (MSE) qui permet de déterminer
le modèle optimal pour analyser les données. Celui-ci se trouve au juste milieu, entre les
modèles victimes de l'un ou de l'autre des phénomènes d'apprentissage. L'erreur quadratique moyenne dépend du biais (*Bias*)
et de la variabilité des mesures.
<!-- #endregion -->

<!-- #region id="a70c00f9" -->
# <a id=quest-ce-que-le-biais-et-la-variance>Qu'est-ce que le biais et la variance?</a>
<!-- #endregion -->

<!-- #region id="e5660c17" -->
Le biais mesure la différence entre le signal idéal et le signal moyen estimé. Supposons que vous avez un
thermomètre et que vous mesurez un grand nombre de fois la température de l'eau bouillante. Vous prenez la moyenne des
mesures et vous obtenez $103 ^\circ C$. Puisque l'eau bout $100 ^\circ C$ (au niveau de la mer) il y a une erreur
systématique de $3 ^\circ C$; c'est le biais. La mesure du thermomètre est biaisée. On peut éliminer ce biais en calibrant
le thermomètre.

La variance mesure la variabilité des résultats. Reprenons l'exemple du thermomètre, maintenant calibré. Si vous
mesurez plusieurs fois la température de l'eau bouillante, vous allez obtenir des valeurs comme suit:
$99,8$$^\circ C$; $100,3$$^\circ C$; $99,5$$^\circ C$; $100,1$$^\circ C$; etc. Ces données ont un écart-type d'environ
$0,3$$^\circ C$ et une variance de $0,09$$^\circ C^2$. Idéalement, l'affichage du thermomètre
indiquerait toujours une température de $100^\circ C$, c'est-à-dire, sans biais ni variabilité dans les mesures.

La figure suivante montrant des cibles de dards résume bien la situation. Le premier panneau montre le cas
d'un thermomètre typique bien ajusté; les données sont bien centrées (erreur moyenne nulle) et
varient peu (variance faible). Le panneau en bas à gauche correspond au thermomètre mal calibré; précis, mais
avec une erreur systématique. Le tableau en haut à droite correspond au thermomètre bien calibré, mais
imprécis, c'est-à-dire, avec une grande variabilité (erreurs de mesures trop grandes). Enfin, le panneau en bas, à droite
correspond à la pire situation; un thermomètre mal calibré et imprécis.

<!-- #endregion -->

<!-- #region id="e6dc2b44" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/biais-variance-diagram.png"  width="400" />
    <div>
    <font size="1.5">Image Source: https://tex.stackexchange.com/questions/307117/reconstructing-the-following-bias-variance-diagram</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="1ff09b30" -->
# <a id=comment-calculer-le-biais-et-la-variance-en-régression>Comment calculer le biais et la variance en régression?</a>
<!-- #endregion -->

<!-- #region id="eaff2c28" -->
Supposons que l'on essaie de mesurer une fonction inconnue $f$

$$y = f(x)$$

et que l'on obtienne des mesures expérimentales

$$y'^{(i)} = f(x^{(i)}) + \text{bruit}^{(i)}$$

L'indice $i$ représente le numéro d'une donnée tel que $1\le i \le n$, où $n$ est le nombre de données. Nous allons recaler un modèle polynomial aux données expérimentales afin d'approximer $f$.
Afin de simplifier l'écriture, on va utiliser la convention suivante pour
décrire un polynôme de degré donné avec les paramètres $\Theta_j$. L'indice $j$ indique que les paramètres ont été
estimés pour un ensemble de données $(x,y')_j$. Le degré du polynôme est constant dans la discussion.

$$\hat{y} = h(x|\Theta_j)$$

Dans ce qui suit, chaque ensemble de données $(x,y')_j$ est séparé en un ensemble d'entraînement et un de test. Un modèle est entraîné avec l'ensemble de test, et les métriques $\text{biais}^2$, $\text{variance}$ et $\text{MSE}$ sont calculées avec l'ensemble de test.

Si l'on génère N ensembles de données bruitées $(x,y')_j$ aux mêmes valeurs de $x$, puis qu'on recale un
polynôme à chaque ensemble, et qu'enfin on calcule la moyenne des N polynômes, on obtient le modèle moyen

$$m(x^{(i)}) = \frac{1}{N}\sum_{j=1} ^{N}h(x^{(i)}|\Theta_j)$$

Le biais mesure la différence entre la fonction inconnue $f$ et le modèle moyen $m$. Idéalement,
il serait nul partout.

$$b(x^{(i)}) = f(x^{(i)}) - m(x^{(i)})$$

On utilise la notation $\text{biais}^2$ pour indiquer la moyenne du carré du biais. On le calcule comme suit:

$$\text{biais}^2 = \frac{1}{n}\sum_{i=1} ^{n}b(x^{(i)})^2$$

La variance des erreurs entre le modèle moyen $m$ et chacun des N polynômes précédents est

$$\text{variance} = \frac{1}{Nn}\sum_{j=1}^{N}\sum_{i=1}^{n}(m(x^{(i)})-h(x^{(i)}|\Theta_j))^2$$

L'erreur quadratique moyenne est définie comme suit
$$\text{MSE} = \frac{1}{Nn}\sum_{j=1}^{N}\sum_{i=1}^{n}(y'^{(i)}-h(x^{(i)}|\Theta_j))^2$$

On peut montrer la relation suivante entre les trois quantités
$$\text{MSE} = \text{biais}^{2} + \text{variance} + \sigma^2$$

où $\sigma^2$ est la variance du bruit gaussien dans les données.

Comme on le verra à plusieurs reprises dans ce module, cette relation permet de décrire les effets du sous-apprentissage et de surapprentissage. Ces problèmes sont omniprésents dans la pratique courante de l'apprentissage automatique. Il est  bien de s'y familiariser tôt afin de comprendre les résultats de nos entraînements.

<!-- #endregion -->

<!-- #region id="1afe87ef" -->
# <a id=la-relation-entre-le-mse-le-biais-et-la-variance>La relation entre le MSE, le biais et la variance</a>
<!-- #endregion -->

<!-- #region id="731af095" -->
Définissons d'abord quelques fonctions qui seront utilisées à plusieurs reprises. Par la suite, nous allons générer le vrai signal idéal que l'on va utiliser pour générer les données. Ce signal sera généré selon la fonction sinusoïdale suivante

$$y = 100\sin(10x).$$
<!-- #endregion -->



In [ ]:
# Génération du signal idéal
def vrai_signal(x):
    y = 100 * np.sin(10 * x)
    return y


# Génération d'échantillons du signal idéal et ajout de bruit gaussien.
# Les valeurs de x sont réparties aléatoirement sur la plage des valeurs
# disponibles.
def genere_signal_bruité(N):
    x = np.random.uniform(x_min, x_max, N)
    y = vrai_signal(x) + np.random.normal(0.0, sigma, N)

    x = x.reshape(-1, 1)
    y = y.reshape(-1, 1)
    return x, y


# Régression à un polynôme de degré donné
def regression_polynome(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))


In [ ]:
# Génération du signal idéal pour des valeurs de x réparties uniformément

x_min = 0
x_max = 1

xx = np.linspace(x_min, x_max, 100)[:, np.newaxis]
yy = vrai_signal(xx)


In [ ]:
# Génération du signal expérimental (c.-à-d. bruité) pour des valeurs de x réparties aléatoirement

sigma = 20  # Niveaux de bruit
N = 20  # Nombres de points du signal
X_data, y_data = genere_signal_bruité(N)


In [ ]:
# Génération du signal idéal pour les mêmes valeurs de x

y_vrai = vrai_signal(X_data)



<!-- #region id="68d6ee4c" -->
Dans ce qui suit, on va générer divers ensembles d'entraînement et de test qui utiliseront toujours les mêmes valeurs de x réparties aléatoirement. On peut donc écrire:
<!-- #endregion -->



In [ ]:
X_train = X_data
X_test = X_data



<!-- #region id="77e46309" -->
# <a id=calcul-du-mse-du-biais-et-de-la-variance>Calcul de le MSE, du $\text{biais}^2$ et de la variance</a>
<!-- #endregion -->

<!-- #region id="4c9100ab" -->
Pour chaque degré de polynôme entre 0 et 12, on génère 1 000 signaux bruités. Chacun est recalé au polynôme
associé. On calcule ensuite l'erreur quadratique moyenne, le $\text{biais}^2$ et la variance pour ces 1 000 ensembles de données.

<!-- #endregion -->



In [ ]:
n_rep = 1000
n_degres = 12
degres = range(n_degres)

hx = np.zeros((N, n_rep))
biais2 = np.zeros(n_degres)
variance = np.zeros(n_degres)
mse = np.zeros(n_degres)

for degre in degres:
    sse = 0
    for n in range(n_rep):
        # Génération de données d'entraînement et de test aux même valeurs de x
        y_train = y_vrai + np.random.normal(0.0, sigma, N).reshape(-1, 1)
        y_test = y_vrai + np.random.normal(0.0, sigma, N).reshape(-1, 1)

        # Entraînement du modèle polynomial avec les données d'entraînement
        reg = regression_polynome(degre).fit(X_train, y_train)

        # Prédiction des valeurs de y pour les données de test
        y_pred = reg.predict(X_test)
        hx[:, n] = np.squeeze(y_pred, axis=1)

        # Erreur de recalage en test
        erreurs = y_pred - y_test

        # Ajout à la somme des carrés des erreurs (SSE)
        sse += np.square(erreurs).mean()

    # Calcule le signal recalé moyen
    E_hx = np.mean(hx, 1).reshape(-1, 1)

    # Calcule le biais^2 entre le signal original et le signal recalé moyen
    biais2[degre] = np.square(E_hx - y_vrai).mean()

    # Calcule la variance entre le signal original et le signal recalé moyen
    variance[degre] = np.square(E_hx - hx).mean()

    # Calcule l'erreur moyenne
    mse[degre] = sse / n_rep


In [ ]:
# Affichage de la relation entre l'erreur quadratique moyenne, le biais et la variance

plt.figure(figsize=(10, 5))
plt.plot(degres, biais2, color="red", label="$biais^{2}$", linestyle="--")
plt.plot(degres, variance, color="blue", label="variance", linestyle="--")
plt.plot(degres, mse, color="black", label="MSE")
plt.legend(loc="best", facecolor="wheat", shadow=True)
plt.xlabel("Degré", fontsize=18)
plt.ylabel("Erreur", fontsize=18)
plt.xlim([1, n_degres])
plt.yscale("log")



<!-- #region id="3f7d4938" -->
On voit que le biais diminue avec le degré du polynôme alors que la variance augmente avec celui-ci. Nous verrons plus loin ce que cela signifie en pratique avec des signaux analysés.
<!-- #endregion -->

<!-- #region id="a27b3989" -->
### <a id=comparaison-des-mesures-du-mse-avec-la-relation-théorique>Comparaison des mesures du MSE avec la relation théorique</a>
<!-- #endregion -->

<!-- #region id="7213bced" -->
La figure suivante montre la superposition des deux courbes.
<!-- #endregion -->



In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(
    degres,
    biais2 + variance + sigma * sigma,
    color="red",
    label="$variance + biais^{2} + \sigma^2$",
    linestyle="-",
    linewidth=10,
    alpha=0.5,
)
plt.plot(degres, mse, color="black", label="MSE")
plt.legend(loc="best", facecolor="wheat", shadow=True)
plt.xlabel("Degré", fontsize=18)
plt.ylabel("Erreur", fontsize=18)
plt.xlim([1, n_degres])
plt.yscale("log")



<!-- #region id="9da027c7" -->
On observe bien la relation théorique

$$\text{MSE} = \text{biais}^{2} + \text{variance} + \sigma^2.$$

<!-- #endregion -->

<!-- #region id="ac3bfd4d" -->
# <a id=quel-est-le-polynôme-recalant-le-mieux-les-données>Quel est le polynôme recalant le mieux les données?</a>
<!-- #endregion -->

<!-- #region id="ed1c0251" -->
C'est celui minimisant l'erreur quadratique moyenne. Déterminons le degré de ce polynôme.
<!-- #endregion -->



In [ ]:
indx = np.argmin(mse)
degre_opt = degres[indx]



<!-- #region id="0278c804" -->
On calcule ensuite les valeurs de la réponse $y$ pour des valeurs de x réparties uniformément.
<!-- #endregion -->



In [ ]:
y_pred_opt = regression_polynome(degre_opt).fit(X_data, y_data).predict(xx)


In [ ]:
# Affichage du vrai signal, du recalage optimal et des données bruitées originales

plt.figure(figsize=(10, 5))
plt.scatter(X_data.ravel(), y_data, color="blue")
lim = plt.axis()
plt.plot(xx, yy, label="vrai signal", color="blue")
plt.plot(xx, y_pred_opt, label="polynôme", color="r")
plt.legend(loc="best", facecolor="wheat", shadow=True)
plt.xlabel("X", fontsize=18)
plt.ylabel("Y", rotation=0, fontsize=18)
plt.title("Polynôme optimal de degré = %d" % (degre_opt))



<!-- #region id="f19583d4" -->
On voit que le polynôme optimal reproduit bien la distribution des données expérimentales ainsi que l'aspect général du vrai signal.

Les prédictions pour des valeurs de x aux extrémités (hors de la plage originale des données (extrapolations)),
sont moins bonnes que celles pour des valeurs de x à l'intérieur (interpolations).
<!-- #endregion -->

<!-- #region id="df41c185" -->
## <a id=comparaison-des-courbes-du-mse-en-entraînement-et-en-test>Comparaison des courbes du MSE en entraînement et en test</a>
<!-- #endregion -->

<!-- #region id="98a4a6e1" -->
Plus le degré d'un polynôme est élevé, plus grande est sa complexité. Dans cette section, nous utilisons
deux ensembles de données bruitées: un ensemble d'entraînement et un autre de test.

Chaque modèle polynomial est entraîné avec les données d'entraînement. On utilise ensuite
le modèle résultant pour prédire les valeurs du signal débruité pour les données d'entraînement et de test.
La valeur du MSE est enfin calculée pour chaque ensemble de données. L'opération est effectuée 1 000
fois afin d'obtenir de meilleures statistiques.
<!-- #endregion -->



In [ ]:
N = 50
n_rep = 1000

mse_train = np.zeros(n_degres)
mse_test = np.zeros(n_degres)

for degre in degres:
    sse_train, sse_test = 0, 0
    for n in range(n_rep):
        # Génération des deux ensembles de données bruitées
        X_train, y_train = genere_signal_bruité(N)
        X_test, y_test = genere_signal_bruité(N)

        # Entraînement du modèle polynomial avec les données d'entraînement
        reg = regression_polynome(degre).fit(X_train, y_train)

        # --- Ensemble d'entraînement ---
        # Prédiction des valeurs du signal débruité
        y_pred = reg.predict(X_train)

        # Prédiction des vraies valeurs du signal
        y = vrai_signal(X_train)

        # Erreurs de prédiction
        erreurs = y_pred - y

        # Cumul de la moyenne des erreurs au carré
        sse_train += np.square(erreurs).mean()

        # --- Ensemble de test ---
        y_pred = reg.predict(X_test)
        y = vrai_signal(X_test)
        erreurs = y_pred - y
        sse_test += np.square(erreurs).mean()

    # Calcul du MSE pour les deux ensembles de données
    mse_train[degre] = sse_train / n_rep
    mse_test[degre] = sse_test / n_rep



<!-- #region id="6e80a039" -->
Affichage des résultats comparant les deux courbes du MSE en fonction de la complexité
du modèle de régression (le degré du polynôme).
<!-- #endregion -->



In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(degres, mse_train, color="#48D1CC", linewidth=3, label="Train")
plt.plot(degres, mse_test, color="red", linewidth=3, label="Test")
plt.legend(loc="best", facecolor="wheat", shadow=True)
plt.xlabel("Degré (Complexité)")
plt.ylabel("Erreur")
plt.xlim([1, n_degres])
plt.yscale("log")



<!-- #region id="3c212e62" -->
La figure montre deux comportements différents du MSE selon que l'on considère les données d'entraînement (*Train*) ou de test.
Discutons d'abord des résultats d'entraînement.

La courbe en vert montre que plus un modèle de régression est complexe, plus le MSE est faible.
Ce n'est pas vraiment surprenant puisqu'il dispose de plus de paramètres à ajuster finement afin de mieux s'adapter à
la distribution des données de test.

La courbe pour les données de test, en rouge, montre que le MSE décroît initialement jusqu'au degré 6, puis
croît par la suite. C'est le degré du polynôme optimal déterminé précédemment.

Les résultats présentés dans la figure dépendent du modèle de recalage utilisé. Ils auraient été différents
avec un autre modèle de régression. Néanmoins, la figure suivante montre l'aspect général des courbes
de MSE qu'on obtient dans la pratique. C'est ce qu'il faut retenir ici.
<!-- #endregion -->

<!-- #region id="5bac3c8e" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/under-vs-over-training.jpeg"  width="600" />
    <div>
    <font size="1.5">Image Source: https://datacadamia.com/data_mining/overfitting</font>
    </div>
</div>

<!-- #endregion -->

<!-- #region id="e9b68175" -->
Le MSE en entraînement diminue graduellement alors que celle de test passe par un minimum avant d'augmenter ensuite.
La courbe de test peut être séparée en deux zones. La zone à gauche du minimum correspond à des modèles trop
simples par rapport à la distribution des données; on y associe le phénomène du sous-apprentissage.

La zone à droite du minimum correspond à des modèles inutilement complexes; c'est le domaine du surapprentissage.
Chaque modèle entraîné contient beaucoup de paramètres pouvant être finement ajustés afin de passer près des données.
On dit alors qu'il apprend à 'mémoriser' la position des données d'entraînement, et donc le bruit dans celles-ci.
Puisque le bruit dans les données de test est différent, les données à recaler sont différentes de celles 'mémorisées'.
Le modèle ne peut les prédire aussi bien que celles d'entraînement. Le problème est d'autant plus important que le modèle
est complexe. Les erreurs de prédiction augmentent, et donc le MSE en test.

La séparation entre les deux zones indique le modèle optimal qui est juste assez complexe pour reproduire
l'aspect des données, mais pas assez pour les « mémoriser ».

La partie suivante explique graphiquement chacun des comportements que l'on vient de décrire.
<!-- #endregion -->

<!-- #region id="2961ce95" -->
# <a id=exemples-de-sous-apprentissage-et-de-surapprentissage-en-régression>Exemples de sous-apprentissage et de surapprentissage en régression</a>
<!-- #endregion -->

<!-- #region id="20fb4a0f" -->
Définissons d'abord quelques fonctions qui seront utilisées à plusieurs reprises
<!-- #endregion -->



In [ ]:
# Fonction affichant le vrai signal et les données expérimentales
def plot_data(ax):
    ax.scatter(X_data, y_data, color="black")
    ax.plot(xx, yy)
    ax.plot([xx.min(), xx.max()], [0, 0], color="black", linestyle="--")
    ax.set_ylim([-150, 150])
    ax.set_xlabel("X", fontsize=14)
    ax.set_ylabel("Y", rotation=0, fontsize=14)
    ax.set_title("Données expérimentales")


# Fonction affichant plusieurs régressions avec un polynôme de degré donné
def fit_poly(degre, N, ax):
    N = 50
    n_rep = 5
    for n in range(n_rep):
        X, y = genere_signal_bruité(N)
        y_pred = regression_polynome(degre).fit(X, y).predict(xx)
        ax.plot(xx, y_pred)

    ax.plot([xx.min(), xx.max()], [0, 0], color="black", linestyle="--")
    ax.set_ylim([-150, 150])
    ax.set_xlabel("X", fontsize=14)
    ax.set_ylabel("Y", rotation=0, fontsize=14)
    ax.set_title("Degré du polynôme = %d" % (degre))



<!-- #region id="d8c4613f" -->
Débutons par générer plusieurs exemples de régression pour des polynômes de degrés variés. Le premier panneau de la figure suivante montre un exemple du signal à recaler avec les divers modèles polynomiaux. La courbe correspond au vrai signal utilisé

$$y = 100\sin(10x)$$

et les points simulent des données expérimentales.
<!-- #endregion -->



In [ ]:
degres = [1, 2, 6, 13, 15]

fig, axes = plt.subplots(2, 3, figsize=(14, 12))
plot_data(axes[0, 0])
fit_poly(degres[0], N, axes[0, 1])
fit_poly(degres[1], N, axes[0, 2])
fit_poly(degres[2], N, axes[1, 0])
fit_poly(degres[3], N, axes[1, 1])
fit_poly(degres[4], N, axes[1, 2])



<!-- #region id="a390d083" -->
## <a id=sous-apprentissage>Sous-apprentissage</a>
<!-- #endregion -->

<!-- #region id="abf095b1" -->
Les résultats pour les polynômes de faibles degrés, 1 et 2, sont trop simples pour reproduire
les détails du signal idéal (courbe en bleu dans le premier panneau). Ils sont tellement
différents du signal original qu'il existe une différence importante entre celui-ci et la moyenne des cinq courbes
dans chaque panneau (non représentée ici, mais facile à visualiser).
La variance des courbes n'est pas très grande. Cette situation (grand biais, faible variance) caractérise
le phénomène du sous-apprentissage. Il est illustré dans le premier panneau de la
figure suivante, montrant à nouveau de diagramme des dards sur une cible.
<!-- #endregion -->

<!-- #region id="3fd7d221" -->
## <a id=surapprentissage>Surapprentissage</a>
<!-- #endregion -->

<!-- #region id="0fba4cd4" -->
Les résultats pour les polynômes de degrés élevés, 13 et 15, reproduisent mieux l'aspect général
du signal idéal. La moyenne des cinq courbes dans chaque panneau serait assez similaire à celui-ci.
Toutefois, les résultats varient grandement aux extrémités de la plage des valeurs de x. Ils ont une
grande variance. Ainsi, les signaux recalés sont plus complexes que le signal original. Cette
situation (faible biais, grande variance) caractérise le phénomène du surapprentissage. Il est
illustré dans le panneau en bas à droite dans la figure suivante.
<!-- #endregion -->

<!-- #region id="0eedff25" -->
## <a id=le-juste-milieu>Le juste milieu</a>
<!-- #endregion -->

<!-- #region id="ee9f5d97" -->
Les résultats pour le polynôme de degré 6 sont les meilleurs. La moyenne des cinq courbes
serait similaire à celle du signal original. Les résultats
ont aussi une faible variabilité que l'on observe aux extrémités de la plage des valeurs de x. Cette
situation, faibles biais et variance, est illustrée dans le panneau en bas à
gauche dans la figure suivante.

Les résultats optimaux observés précédemment au minimum de la courbe de l'erreur quadratique moyenne en test
(polynôme de degré 6) correspondent bien à la situation idéale où tous les dards se
trouveraient concentrés au centre de la cible.
<!-- #endregion -->

<!-- #region id="e32c7d0b" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/overfitting-illustration.jpeg"  width="300" />
    <div>
    <font size="1.5">Image Source: https://datacadamia.com/data_mining/overfitting</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="f702f597" -->
# <a id=le-sous-apprentissage-et-le-surapprentissage-en-classification>Le sous-apprentissage et le surapprentissage en classification</a>
<!-- #endregion -->

<!-- #region id="319cc225" -->
Les deux phénomènes sont également rencontrés en classification. La figure suivante en montre des
exemples en classification binaire, c'est-à-dire, où il n'y a que deux classes. On observe ceci:

- le panneau de gauche montre un cas de sous-apprentissage. La frontière linéaire entre les deux classes est
trop simple pour reproduire la distribution des données de chaque classe. Il en résulte plusieurs erreurs de
classification (8 croix et 1 cercle),
- le panneau de droite montre un cas de surapprentissage. Bien qu'elle ne résulte en aucune erreur
de classification, la frontière entre les deux classes est trop complexe pour être crédible. Lors
de son entraînement, le classificateur à fini par mémoriser la classe de chaque point afin de bien le classifier,
- le panneau central montre le meilleur compromis. La frontière, en arc de cercle, entre les deux
classes reproduit assez bien, mais pas parfaitement, la distribution des données de chaque classe. En effet,
il demeure quelques erreurs de classification (1 croix et 1 cercle).
<!-- #endregion -->

<!-- #region id="8b79f0fb" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/biais-variance-illustration.png"  width="700" />
    <div>
    <font size="1.5">Image Source: https://x-wei.github.io/Ng_DLMooc_c2wk1.html</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="275a2a9d" -->
Chaque frontière de décision résulte de l'entraînement d'un classificateur avec les données d'entraînement.
Si l'on refaisait les entraînements précédents avec de nouvelles données, réparties aléatoirement de
façons similaires, on observerait ceci:

- panneau de gauche: la frontière linéaire aurait une orientation différente, mais serait toujours
aussi mauvaise à séparer les deux classes.

- panneau de droite: la séparation des courbes serait à nouveau parfaite, toujours aussi complexe,
mais avec des détails très différents. Ce n'est pas réaliste.

- panneau central: c'est la frontière qui aurait le moins changé. Il y aurait encore
des erreurs de classification, mais la stabilité de la frontière indiquerait que c'est la
plus robuste face aux changements de données. C'est ce que l'on recherche chez un classificateur; un minimum
d'erreurs de classification avec une frontière variant peu avec de nouvelles données.
<!-- #endregion -->
